In [1]:
import os
import sys

import math
import time
import datetime
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view
import torch
from torch.utils.data import Dataset
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torch.optim.lr_scheduler import CosineAnnealingLR
from neuralop.models import TFNO3d


# from YourDataset import YourDataset  # Import your custom dataset here
from tqdm import tqdm
from torch.cuda.amp import autocast, GradScaler

import pickle

torch.manual_seed(23)

scaler = GradScaler()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


/tmp/ipykernel_3873133/2891255144.py:26: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [2]:
# Define your custom loss function here
class CustomLoss(nn.Module):
    def __init__(self):
        super(CustomLoss, self).__init__()

    def forward(self, y_pred, y_true, Par, Lambda=None):
        # Implement your custom loss calculation here
        if Lambda is not None:
            residue = torch.absolute(y_true - y_pred)
            Lambda = Par['gamma']*Lambda + Par['eta']*residue/torch.max(residue)
            loss = torch.mean(torch.square(Lambda*residue)) 
        
        else:
            loss = torch.mean(torch.square(y_true - y_pred)) 
            # loss = torch.norm(y_true-y_pred, p=2)/torch.norm(y_true, p=2)

        return loss, Lambda

class YourDataset(Dataset):
    def __init__(self, x, y, transform=None):
        self.x = x
        self.y = y
        self.transform = transform

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        x_sample = self.x[idx]
        y_sample = self.y[idx]

        if self.transform:
            x_sample, y_sample = self.transform(x_sample, y_sample)

        return x_sample, y_sample

class Normalizer():
    def __init__(self, shift, scale):
        self.shift = torch.tensor(shift, dtype=torch.float32)
        self.scale = torch.tensor(scale, dtype=torch.float32)
    
    def normalize(self, x):
        return (x - self.shift)/self.scale
    
    def renormalize(self, x):
        return x*self.scale + self.shift

def get_xyt_grid(nx=None, ny=None, nt=None, bot=[0, 0, 0], top=[1, 1, 1], dtype='float32',
                 x_arr=None, y_arr=None, t_arr=None, dt=0.):
    '''
    Args:
        S: number of points on each spatial domain
        T: number of points on temporal domain including endpoint
        bot: list or tuple, lower bound on each dimension
        top: list or tuple, upper bound on each dimension

    Returns:
        (n_x, n_y, n_t, 3) array of grid points
    '''
    if x_arr is None:
        x_arr = np.linspace(bot[0], top[0], num=nx, endpoint=True)

    if y_arr is None:
        y_arr = np.linspace(bot[1], top[1], num=ny, endpoint=True)

    if t_arr is None:
        if dt is None:
            dt = (top[2] - bot[2]) / nt
        t_arr = np.linspace(bot[2] + dt, top[2], num=nt)

    x_grid, y_grid, t_grid = np.meshgrid(x_arr, y_arr, t_arr, indexing='ij')
    x_axis = np.ravel(x_grid)
    y_axis = np.ravel(y_grid)
    t_axis = np.ravel(t_grid)
    grid = np.stack([x_axis, y_axis, t_axis], axis=0).T

    x_grid, y_grid = np.meshgrid(x_arr, y_arr, indexing='ij')
    x_axis = np.ravel(x_grid)
    y_axis = np.ravel(y_grid)
    grid_space = np.stack([x_axis, y_axis], axis=0).T

    grids_dict = {'grid_x': x_arr, 
                  'grid_y': y_arr,
                  'grid_t': t_arr, 
                  'grid_space': grid_space, 
                  'grid': grid}
    for key in grids_dict.keys():
        grids_dict[key] = torch.tensor(grids_dict[key], dtype=eval('torch.' + dtype), device='cpu')
    return grids_dict



def _preprocess_fno(data_dict, Par):
    Grid_train = get_xyt_grid(Par['nx'], Par['ny'], Par['lf'], bot=[0, 0, 0], top=[1, 1, 1], dtype='float32')
    Grid_test  = get_xyt_grid(Par['nx'], Par['ny'], Par['lf'], bot=[0, 0, 0], top=[1, 1, 1], dtype='float32')
    
    data_dim = len(data_dict['x_train'].shape) - 2
    if data_dim != len(data_dict['x_test'].shape) - 2:
        raise ValueError('Data dimension mismatch between train and test sets.' +
                            f'Train: {data_dim}, Test: {len(data_dict["x_test"].shape)}')

    # Repeat shape is [1, time_steps_train, 1, 1, 1] for 3D data
    time_steps_train = Par['lf'] #self.configs.time_steps_train
    time_steps_val = Par['lf'] #self.configs.time_steps_inference if self.configs.scenario == 'hypersonics' \
    time_steps_test = Par['lf'] #self.configs.time_steps_inference
    repeat_shape_train = [1, time_steps_train] + [1] * data_dim
    repeat_shape_val = [1, time_steps_val] + [1] * data_dim
    repeat_shape_test = [1, time_steps_test] + [1] * data_dim

    data_dict['x_train'] = data_dict['x_train'].repeat(repeat_shape_train)
    data_dict['x_val'] = data_dict['x_val'].repeat(repeat_shape_val)
    data_dict['x_test'] = data_dict['x_test'].repeat(repeat_shape_test)

  

    for dataset in ['x_train', 'x_val', 'x_test']:
        n_samples = data_dict[dataset].shape[0]
        if True:
            grid = Grid_test if dataset == 'x_test' else Grid_train
            time_steps = time_steps_test if dataset == 'x_test' else time_steps_train

        grid_t = grid['grid_t'].reshape([1, time_steps] + [1] * data_dim)
        grid_t = grid_t.repeat(
            [n_samples, 1] + list(data_dict[dataset].shape[2:]))
        if data_dim == 1:
            x_shape = [1, 1] + list(data_dict[dataset].shape[2:3])
            grid_x = grid['grid_x'].reshape(
                x_shape).repeat(n_samples, time_steps, 1)
            data_dict[dataset] = torch.stack(
                [data_dict[dataset], grid_x, grid_t], dim=1)
        elif data_dim == 2:
            x_shape = [1, 1] + list(data_dict[dataset].shape[2:3]) + [1]
            y_shape = [1, 1, 1] + list(data_dict[dataset].shape[3:])

            grid_x = grid['grid_x'].reshape(x_shape).repeat(
                n_samples, time_steps, 1, y_shape[-1])
            grid_y = grid['grid_y'].reshape(y_shape).repeat(
                n_samples, time_steps, x_shape[-2], 1)

            data_dict[dataset] = torch.stack(
                [data_dict[dataset], grid_x, grid_y, grid_t], dim=1)
        elif data_dim == 3:
            x_shape = [1, 1] + list(data_dict[dataset].shape[2:3]) + [1, 1]
            y_shape = [1, 1, 1] + list(data_dict[dataset].shape[3:4]) + [1]
            z_shape = [1, 1, 1, 1] + list(data_dict[dataset].shape[4:])
            grid_x = grid['grid_x'].reshape(x_shape)
            grid_y = grid['grid_y'].reshape(y_shape)
            grid_z = grid['grid_z'].reshape(z_shape)

            grid_x = grid_x.repeat(
                n_samples, time_steps, 1, y_shape[-2], z_shape[-1])
            grid_y = grid_y.repeat(
                n_samples, time_steps, x_shape[-3], 1, z_shape[-1])
            grid_z = grid_z.repeat(
                n_samples, time_steps, x_shape[-3], y_shape[-2], 1)

            data_dict[dataset] = torch.stack(
                [data_dict[dataset], grid_x, grid_y, grid_z, grid_t], dim=1)
    return data_dict


def preprocess(traj, Par):
    x = sliding_window_view(traj[:,:-(Par['lf']-1),:,:], window_shape=Par['lb'], axis=1 ).transpose(0,1,4,2,3).reshape(-1,Par['lb'],Par['nx'], Par['ny'])
    y = sliding_window_view(traj[:,Par['lb']-1:,:,:], window_shape=Par['lf'], axis=1 ).transpose(0,1,4,2,3).reshape(-1,Par['lf'],Par['nx'], Par['ny'])
    t = np.linspace(0,1,Par['lf']).reshape(-1,1)

    nt = y.shape[1]
    n_samples = y.shape[0]

    # t = np.tile(t, [n_samples,1]).reshape(-1,)                                                     #[_*nt, ]
    # x = np.repeat(x,nt, axis=0)                                   #[_*nt, 1, 64, 64]
    # y = y.reshape(y.shape[0]*y.shape[1],1,y.shape[2],y.shape[3])  #[_*nt, 64, 64]


    print('x: ', x.shape)
    print('y: ', y.shape)
    print('t: ', t.shape)
    print()
    return x,y,t


In [3]:
# Load your data into NumPy arrays (x_train, t_train, y_train, x_val, t_val, y_val, x_test, t_test, y_test)
#########################
x = np.load('../data/x.npy')  #[_, 64, 64]
t = np.load('../data/t.npy')  #[_, 200]
y = np.load('../data/y.npy')  #[_, 64, 64, 200]

traj = np.append( np.expand_dims(x, axis=-1), y, axis=-1 ).transpose(0,3,1,2) #[_, 64, 64, 201]

traj_train = traj[:800, ::4]
traj_val   = traj[800:900, ::4]
traj_test  = traj[900:, ::4]

print()

Par = {}
# Par['nt'] = 100 
Par['nx'] = traj_train.shape[2]
Par['ny'] = traj_train.shape[3]
Par['nf'] = 1
Par['lb'] = 1
Par['lf'] = 51 
# Par['temp'] = Par['nt'] - Par['lb'] - Par['lf'] + 2

Par['num_epochs'] = 500

print('\nTrain Dataset')
x_train, y_train, t_train = preprocess(traj_train, Par)
print('\nValidation Dataset')
x_val, y_val, t_val  = preprocess(traj_val, Par)
print('\nTest Dataset')
x_test, y_test, t_test  = preprocess(traj_test, Par)

t_min = np.min(t_train)
t_max = np.max(t_train)

Par['inp_shift'] = np.mean(x_train) 
Par['inp_scale'] = np.std(x_train)
Par['out_shift'] = np.mean(y_train)
Par['out_scale'] = np.std(y_train)
Par['t_shift']   = t_min
Par['t_scale']   = t_max - t_min

Par['eta']   = 0.1
Par['gamma'] = 0.99

Par['Lambda_max'] = Par['eta']/(1 - Par['gamma'])

Lambda = np.ones(y_train.shape, dtype=np.float32)*Par['Lambda_max']/2.0
print("Lambda: ", Lambda.shape)

inp_normalizer = Normalizer(Par['inp_shift'], Par['inp_scale'])
out_normalizer = Normalizer(Par['out_shift'], Par['out_scale'])

Par['model'] = 'FNO'

with open('Par.pkl', 'wb') as f:
    pickle.dump(Par, f)

#########################

# Create custom datasets
x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
t_train_tensor = torch.tensor(t_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
Lambda_tensor = torch.tensor(Lambda, dtype=torch.float32)

x_val_tensor   = torch.tensor(x_val,   dtype=torch.float32)
t_val_tensor   = torch.tensor(t_val,   dtype=torch.float32)
y_val_tensor   = torch.tensor(y_val,   dtype=torch.float32)

x_test_tensor  = torch.tensor(x_test,  dtype=torch.float32)
t_test_tensor  = torch.tensor(t_test,  dtype=torch.float32)
y_test_tensor  = torch.tensor(y_test,  dtype=torch.float32)

data_dict = {'x_train':x_train_tensor, 'x_val':x_val_tensor, 'x_test':x_test_tensor}
data_dict = _preprocess_fno(data_dict, Par)

x_train_tensor = torch.tensor(data_dict['x_train'].cpu().numpy(), dtype=torch.float32)
x_val_tensor   = torch.tensor(data_dict['x_val'].cpu().numpy()  , dtype=torch.float32)
x_test_tensor  = torch.tensor(data_dict['x_test'].cpu().numpy() , dtype=torch.float32)

# train_dataset = YourDataset(x_train_tensor, y_train_tensor)
# val_dataset = YourDataset(x_val_tensor, y_val_tensor)
# test_dataset = YourDataset(x_test_tensor, y_test_tensor)

# Define data loaders
train_batch_size = 10
val_batch_size   = 10
test_batch_size  = 10
# train_loader = DataLoader(train_dataset, batch_size=train_batch_size)
# val_loader = DataLoader(val_dataset, batch_size=val_batch_size)
# test_loader = DataLoader(test_dataset, batch_size=test_batch_size)





Train Dataset
x:  (800, 1, 64, 64)
y:  (800, 51, 64, 64)
t:  (51, 1)


Validation Dataset
x:  (100, 1, 64, 64)
y:  (100, 51, 64, 64)
t:  (51, 1)


Test Dataset
x:  (100, 1, 64, 64)
y:  (100, 51, 64, 64)
t:  (51, 1)

Lambda:  (800, 51, 64, 64)


# Loading the model

In [4]:
# Initialize your Unet2D model
if Par['model']=='FNO':
    model = TFNO3d(12, 12, 12, hidden_channels=20, in_channels=4, out_channels=1).cuda()
print('Created '+Par['model'])
METHOD='logarithmic'
print('Loading model ...')
path_model = f'models/best_model_{METHOD}_{METHOD}.pt'
model.load_state_dict(torch.load(path_model,weights_only=False))

# Define loss function and optimizer
criterion = CustomLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)

Created FNO
Loading model ...


# Sanity Check

In [5]:
# Validation loop
model.eval()
val_loss = 0.0
with torch.no_grad():
    for start in range(0, x_val.shape[0]-1, val_batch_size):
        end = start + val_batch_size
        x = x_val_tensor[start:end]
        y_true = y_val_tensor[start:end]
        x = inp_normalizer.normalize(x)
        y_pred = out_normalizer.renormalize(model(x.to(device))[:,0]) 
        loss, _= criterion(y_pred, y_true.to(device), Par)
        val_loss += loss.item()

val_loss /= int(y_val.shape[0]/val_batch_size)
print(f'Val Loss: {val_loss:.4e}')

Val Loss: 3.0454e-04


In [6]:
y_true_ls = []
y_pred_ls = []

# Testing loop
model.eval()
test_loss = 0.0
with torch.no_grad():
    for start in range(0, x_test.shape[0]-1, test_batch_size):
        end = start + test_batch_size
        x = x_test_tensor[start:end]
        y_true = y_test_tensor[start:end]
        x = inp_normalizer.normalize(x)
        y_pred = out_normalizer.renormalize(model(x.to(device))[:,0]) 
        loss, _= criterion(y_pred, y_true.to(device), Par)
        y_true_ls.append(y_true)
        y_pred_ls.append(y_pred)
        test_loss += loss.item()

test_loss /= int(y_test.shape[0]/test_batch_size)
print(f'Test Loss: {test_loss:.4e}')

# # Testing loop
# model.eval()
# test_loss = 0.0
# with torch.no_grad():
#     for x, y_true in test_loader:
#         with autocast():
#             y_pred = model(x.to(device), X_loc_test_tensor.to(device))
#             loss = criterion(y_pred, y_true.to(device), Par).item() 
#         y_true_ls.append(y_true)
#         y_pred_ls.append(y_pred)
#         test_loss += loss 
# test_loss /= len(test_loader)
# print(f'Test Loss: {test_loss:.4e}')

Y_TRUE = torch.cat(y_true_ls, axis=0)
Y_PRED = torch.cat(y_pred_ls, axis=0)

print('true: ', Y_TRUE.shape)
print('pred: ', Y_PRED.shape)

Test Loss: 3.5704e-04
true:  torch.Size([100, 51, 64, 64])
pred:  torch.Size([100, 51, 64, 64])


In [7]:
Y_TRUE = Y_TRUE.detach().cpu().numpy()
Y_PRED = Y_PRED.detach().cpu().numpy()

In [8]:
np.save("Y_TRUE.npy", Y_TRUE)
np.save("Y_PRED.npy", Y_PRED)

In [9]:
true = np.load("Y_TRUE.npy")
pred = np.load("Y_PRED.npy")
true.shape, pred.shape, np.mean( (true -pred)**2 )

((100, 51, 64, 64), (100, 51, 64, 64), 0.0003570364)